In [10]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib as plot
import math
import talib
from sklearn import metrics, linear_model, preprocessing, model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.externals import joblib

In [2]:
conn = sqlite3.connect('poloniex_0.1.db')
c = conn.cursor()

In [3]:
df = pd.read_sql_query("SELECT * FROM candles_USDT_ETH", conn)
df = df.drop('id' ,1)
df = df.drop('trades' ,1)
df['start'] = pd.to_datetime(df['start'],unit='s')
df.set_index('start' , inplace = True)
#df.index.dtype
df.head(10)
#print (len(df.index))


,open,high,low,close,vwp,volume
start,,,,,,
2017-06-12 14:37:00,376.500010,376.500010,375.423819,376.50000,375.984977,6.203722
2017-06-12 14:38:00,376.435997,376.480000,375.000000,376.40001,376.234584,13.728674
2017-06-12 14:39:00,375.000000,376.000000,374.105980,376.00000,375.278772,11.562601
2017-06-12 14:40:00,375.657617,376.000000,374.105980,376.00000,375.938314,6.251341
2017-06-12 14:41:00,376.400000,376.600000,376.000100,376.60000,376.448518,63.735705
2017-06-12 14:42:00,376.000100,376.600000,376.000000,376.00000,376.200211,6.293728
2017-06-12 14:43:00,376.600000,376.600000,375.000000,375.00000,375.729590,147.057915
2017-06-12 14:44:00,376.600000,376.909634,376.000000,376.01014,376.259007,143.724666
2017-06-12 14:45:00,376.010140,377.962200,376.010140,377.96220,376.243165,9.771476


In [4]:
def safediv(x,y):
    if y == 0 :
        return np.nan
    else:
        return x/y

In [5]:
def ohlcsum(df):
    #df = df.sort()
    
    return {
       'open': df['open'][0],
       'high': df['high'].max(),
       'low': df['low'].min(),
       'close': df['close'][-1],
       'volume': df['volume'].sum(),
        
        'vwp': safediv((df['vwp']* df['volume']).sum(),df['volume'].sum()) 
        
      }

In [6]:
def convertcandles(df, mins):
    s = str(mins) + 'T'
    return( df.resample(s).agg(ohlcsum))
    
    
#convertcandles(df, 120)
    

In [14]:
newdf = convertcandles(df,60)

#newdf = df
price = np.array(newdf['vwp'])
h = np.array(newdf['high'])
l = np.array(newdf['low'])
c = np.array(newdf['close'])
#sma  = talib.SMA(price,50)
m = talib.MACD(price, 12,26,9)

mom = talib.MOM(price,12 )
rsi = talib.RSI(price)
krsi, drsi = talib.STOCHRSI(price)
roc = talib.ROC(price)
mean5d = pd.rolling_mean(newdf['vwp'],5)
mean10d = pd.rolling_mean(newdf['vwp'],10)
newdf['willR'] = talib.WILLR(h,l,c)
newdf['krsi'] = krsi
newdf['drsi'] = drsi
newdf['disp5d'] = newdf['vwp'] - mean5d
newdf['disp10d'] = newdf['vwp'] - mean10d
newdf['cci'] = talib.CCI(h,l,c)
newdf['roc'] = roc
#m[1][40:60]
# newdf['msig'] = np.where((m[1]>m[2]),1,0)
newdf['msig'] = m[1]-m[2]
#newdf['mom'] = np.where(mom>0,1,0)
# newdf['rsi'] = np.where((rsi>50)& (rsi> np.roll(rsi,1)),1,0)
newdf['rsi'] = rsi
# newdf['sma'] = np.where((pd.rolling_mean(newdf['vwp'],9)- pd.rolling_mean(newdf['vwp'],26))>0,1,0)
newdf['target'] = np.where((newdf['close'].shift(1)- newdf['close'])>0,1,0)
newdf.dropna(axis = 0, how = 'any', inplace = True)
#newdf.drop(newdf.index[0], inplace = True)
newdf.head(30)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
  from ipykernel import kernelapp as app
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=10,center=False).mean()
  app.launch_new_instance()


,open,high,low,close,vwp,volume,willR,krsi,drsi,disp5d,disp10d,cci,roc,msig,rsi,target
start,,,,,,,,,,,,,,,,
2017-06-13 23:00:00,376.100000,382.336897,375.200000,382.300000,379.845738,1683.212360,-20.537954,3.120716,7.589479e+00,-3.750856,-0.459379,38.188076,3.548567,5.540354,52.105004,0
2017-06-14 00:00:00,382.300000,383.900000,379.500000,380.500000,381.248085,699.032086,-30.075123,17.704987,6.941901e+00,-1.292843,-0.207657,43.959307,3.111966,5.782345,52.892583,1
2017-06-14 01:00:00,380.500000,382.019720,377.000000,381.378362,378.989983,2312.091275,-28.851526,0.000000,6.941901e+00,-1.529865,-2.716431,23.590376,0.665825,6.046305,51.426155,0
2017-06-14 02:00:00,381.400000,382.000000,378.327335,379.000070,380.041085,690.325339,-43.841319,45.404545,2.103651e+01,0.110304,-1.911018,5.846718,0.650688,6.102000,52.091980,1
2017-06-14 03:00:00,379.000140,388.000000,378.145420,381.700000,384.773781,1299.738475,-37.300886,100.000000,4.846818e+01,3.794046,2.551415,46.217364,0.707362,5.850889,55.077822,0
2017-06-14 04:00:00,381.700000,383.994523,378.654086,382.840000,381.992796,966.418135,-38.974041,42.770533,6.272503e+01,0.583650,-0.510075,12.492914,0.739750,5.842442,52.987993,0
2017-06-14 05:00:00,381.255493,385.000000,380.432000,380.432000,382.429786,636.173292,-58.803178,50.984058,6.458486e+01,0.784300,0.336579,5.295369,-1.059858,5.774827,53.287923,1
2017-06-14 06:00:00,380.737705,381.512000,375.551150,376.500000,378.827224,1569.376215,-85.278788,0.000000,3.125153e+01,-2.785710,-2.239167,-125.484880,-2.638982,5.860828,50.431401,1
2017-06-14 07:00:00,376.500001,377.352150,369.000000,369.500000,372.598449,1779.306649,-97.731421,0.000000,1.699469e+01,-7.525958,-7.429145,-226.615643,-2.712361,6.093467,45.854547,1


In [20]:
factors = ['volume','msig','rsi','cci','willR','krsi','drsi','disp5d','disp10d']
X = newdf[factors]
Y = newdf['target']

In [23]:
Xtrain,  Xtest, Ytrain, Ytest = model_selection.train_test_split(X,Y)
#print(Xtrain.head())
scaler = preprocessing.MinMaxScaler(copy = False)
scaler.fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
mod = linear_model.LogisticRegression(solver = "saga")
mod.fit(Xtrain, Ytrain)
scaler_filename = "scalereth60.save"
joblib.dump(scaler, scaler_filename) 

Xtest =scaler.transform(Xtest)
print('LogisticRegression score: %f'
      % mod.score(Xtest, Ytest))

LogisticRegression score: 0.678930


In [26]:
# names = [ "Linear SVM", "RBF SVM", "Gaussian Process",
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes"]
names = [ "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost","LogisticRegression"
         ]
# classifiers = [SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1),
#     AdaBoostClassifier(),
#     GaussianNB()]
classifiers = [SVC(kernel="linear", C=0.025,probability=True),
    SVC(gamma=2, C=1,probability=True),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),LogisticRegression(solver = "saga")]
for name, clf in zip(names, classifiers):
    clf.fit(Xtrain, Ytrain)
    clf_probs = clf.predict_proba(Xtest)
    logscore = metrics.log_loss(Ytest, clf_probs)
    score = clf.score(Xtest, Ytest)
    print(name, score, logscore)

('Linear SVM', 0.69230769230769229, 0.60374863257213651)
('RBF SVM', 0.70903010033444813, 0.56317754823586752)
('Decision Tree', 0.68896321070234112, 1.1531235746711945)
('Random Forest', 0.67892976588628762, 0.61074093017251163)
('Neural Net', 0.68227424749163879, 0.58004360739632654)
('AdaBoost', 0.73578595317725748, 0.64987592162114416)
('LogisticRegression', 0.67892976588628762, 0.57428947146947851)


In [24]:
clf  = model_selection.GridSearchCV(SVC(probability=True),cv = 5, param_grid= {'C': np.logspace(-3, 2, 6), 'gamma': np.logspace(-3, 2, 6)})
clf.fit(Xtrain, Ytrain)
# clf_probs = clf.predict_proba(Xtest)
# logscore = metrics.log_loss(Ytest, clf_probs)
# score = clf.score(Xtest, Ytest)
#print( model.best_estimator_ )
print( clf.best_score_ )
print( clf.best_params_ )
# print(score, logscore)

0.748049052397
{'C': 100.0, 'gamma': 0.10000000000000001}


In [25]:

clf = SVC(gamma=0.1, C=100,probability=True)
clf.fit(Xtrain, Ytrain)
filename = 'finalized_model_eth60.sav'
joblib.dump(clf, filename)

['finalized_model_eth60.sav']